[Rendered Version](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/Training%20Doc.ipynb)

In [1]:
from taxcalc import *

This example uses the packaged CPS file. If you were to use the PUF, you would create the records class using
`Records()` or `Records('path to puf')` if the PUF is not in your current working directory.

In [2]:
# Initiate baseline calculator
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [3]:
# Inititate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


You can specify your reforms in a JSON file and, using the `read_json_param_objects` method, create a dictionary containing both policy reforms and behavioral assumptions

The policy reform and any behavioral assumptions need to be in separate JSON files.

In [5]:
reforms = calc_x.read_json_param_objects('SampleReform.json', 'SampleBehavior.json')
reforms

{'behavior': {2018: {'_BE_inc': array([-0.25])}},
 'consumption': {2018: {'_MPC_e19800': array([ 0.2])}},
 'growdiff_baseline': {},
 'growdiff_response': {},
 'policy': {2018: {'_ID_RealEstate_hc': array([ 0.25]),
   '_ID_StateLocalTax_hc': array([ 0.25]),
   '_II_rt1': array([ 0.12]),
   '_II_rt2': array([ 0.12]),
   '_II_rt7': array([ 0.35]),
   '_STD': array([[12000, 24000, 12000, 18000, 24000]])},
  2019: {'_ID_RealEstate_hc': array([ 0.5]),
   '_ID_StateLocalTax_hc': array([ 0.5])},
  2020: {'_ID_RealEstate_hc': array([ 0.75]),
   '_ID_StateLocalTax_hc': array([ 0.75])},
  2021: {'_ID_RealEstate_hc': array([ 1.]),
   '_ID_StateLocalTax_hc': array([ 1.])}}}

Alternatively, you can specify a policy reform as a dictionary. The only drawback with doing this is you will not be able to use the `reform_documentation()` method to print out the reform description.

In [6]:
reform_dict = {
    2018: {
        '__ID_RealEstate_hc': [0.25],
        '_ID_StateLocalTax_hc': [0.25],
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt7': [0.35],
        '_STD': [[12000, 24000, 12000, 18000, 24000]]},
    2019: {
        '_ID_RealEstate_hc': [0.5],
        '_ID_StateLocalTax_hc': [0.5]},
    2020: {
        '_ID_RealEstate_hc': [0.75],
        '_ID_StateLocalTax_hc': [0.75]},
    2021: {
        '_ID_RealEstate_hc': [1.],
        '_ID_StateLocalTax_hc': [1.]}
}

In [7]:
calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [8]:
calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)

### Viewing the results

In [9]:
from taxcalc.utils import *

#### Analyzing Individual Variables

Individual variables are attributes of the records class and can therefore be accessed using a simple dot notation

In [10]:
baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

Combined Liability - Baseline: 2551551433353.72
Combined Liability - Reform:   2435921968154.79
-----------------------------------------------
Difference:                    -115629465198.93


#### Diagnostic Table

Diagnostic tables are the most straight forward methods of evaluation. They simply show aggregate values for a given calculator

In [11]:
create_diagnostic_table(calc)

,2018
Returns (#m),169.9
AGI ($b),"10,557.3"
Itemizers (#m),67.9
Itemized Deduction ($b),"1,491.9"
Standard Deduction Filers (#m),84.0
Standard Deduction ($b),772.4
Personal Exemption ($b),"1,221.6"
Taxable Income ($b),"7,484.1"
Regular Tax ($b),"1,402.3"
AMT Income ($b),"9,722.1"


In [12]:
create_diagnostic_table(calc_response)

,2018
Returns (#m),169.9
AGI ($b),"10,529.9"
Itemizers (#m),28.8
Itemized Deduction ($b),799.3
Standard Deduction Filers (#m),123.1
Standard Deduction ($b),"2,120.9"
Personal Exemption ($b),"1,221.6"
Taxable Income ($b),"7,101.9"
Regular Tax ($b),"1,289.5"
AMT Income ($b),"10,038.0"


#### Distribution Table

The distribution table shows the same information as the diagnostic table, but broken down by income bin or decile. You can view the results as either the weighted average or the weighted sum in each bin

In [13]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,-220,"16,988,849",-648,"9,521,828","6,576","11,440",5,"5,251",25,1,-651,0,0,1,0,0,98,-97,176,79
1,"9,909","16,990,011","5,158","11,490,962","7,808","299,484",217,"5,914",190,13,"5,055","9,099",0,13,3,0,760,-750,666,-84
2,"17,325","16,989,299","9,228","11,844,173","7,834","1,575,320","1,053","6,322","1,670",157,"8,624","24,141",1,158,30,0,"1,031",-903,"1,229",325
3,"25,294","16,988,882","17,002","12,299,405","7,615","3,433,563","2,320","7,146","5,063",534,"15,612","27,233",2,535,90,0,"1,190",-745,"2,329","1,584"
4,"34,370","16,989,771","26,122","11,419,346","7,168","5,139,906","3,865","7,647","10,783","1,239","23,779","34,780",4,"1,243",167,0,917,159,"3,588","3,747"
5,"45,872","16,989,771","37,776","10,041,293","6,394","6,843,169","5,940","8,103","19,093","2,283","34,190","28,670",5,"2,288",278,0,430,"1,579","5,163","6,743"
6,"61,673","16,989,779","54,284","7,678,091","4,899","9,292,705","9,505","8,496","32,023","4,163","48,512","28,421",3,"4,166",392,0,83,"3,691","7,359","11,050"
7,"85,391","16,988,988","78,311","5,250,777","3,402","11,735,439","13,927","9,338","51,881","7,499","70,163","29,298",5,"7,504",495,0,13,"6,995","10,717","17,712"
8,"123,840","16,989,812","114,469","3,055,434","2,045","13,931,651","19,638","10,527","82,353","12,878","103,334","48,809",6,"12,884",453,0,2,"12,428","15,796","28,224"
9,"295,123","16,989,678","279,699","1,373,913",989,"15,615,406","31,341","10,012","237,430","53,772","263,621","3,887,852",948,"54,720",29,518,0,"55,209","25,593","80,802"


In [14]:
create_distribution_table(calc_response.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"16,988,849","-11,015,996,775","9,521,828","195,166,315,775","11,440","84,787,078","89,205,327,406","418,135,089","15,130,228","-11,058,378,844",0,0,"15,130,228","7,547,086",0,"1,662,393,337","-1,654,810,195","2,995,731,088","1,340,920,894"
1,"16,989,913","87,755,237,374","11,786,854","240,265,337,808","14,332","132,944,199","100,431,351,368","1,169,095,676","37,881,772","87,707,306,885","9,099","5,039,021","42,920,793","22,355,780",0,"12,950,024,022","-12,929,459,009","11,341,553,724","-1,587,905,285"
2,"16,988,685","156,530,970,290","13,237,496","253,380,353,374","175,315","3,363,715,728","107,239,369,277","5,670,011,305","576,436,104","154,347,006,581","29,369","16,860,427","593,296,531","112,098,702",0,"17,768,990,850","-17,287,793,021","20,831,239,053","3,543,446,032"
3,"16,989,649","288,207,853,657","15,137,261","268,595,750,846","596,375","10,848,588,329","121,231,899,523","47,320,767,894","5,325,104,502","281,260,276,701","33,065","39,171,732","5,364,276,234","836,329,221",0,"20,919,934,989","-16,391,987,976","39,515,966,488","23,123,978,512"
4,"16,990,272","442,397,388,978","15,597,422","278,780,640,805","958,410","17,566,514,179","130,125,065,406","122,899,934,515","13,983,246,808","430,920,257,132","36,679","78,483,942","14,061,730,750","1,811,872,243",0,"16,701,418,917","-4,451,560,411","60,812,178,900","56,360,618,490"
5,"16,989,273","638,866,555,448","15,345,517","280,736,934,648","1,540,827","31,951,309,403","137,695,992,595","245,051,263,731","28,032,179,549","618,157,347,893","31,902","99,995,843","28,132,175,391","3,263,159,714",0,"8,887,945,698","15,981,069,979","87,387,024,204","103,368,094,183"
6,"16,989,016","917,636,371,645","14,351,284","274,994,236,812","2,615,620","57,564,940,870","144,325,062,149","457,621,094,421","54,607,322,593","880,917,571,846","37,252","106,357,308","54,713,679,901","6,014,564,294",0,"2,052,689,031","46,646,426,577","124,331,623,491","170,978,050,068"
7,"16,990,068","1,325,805,360,155","12,932,406","268,468,206,638","4,054,971","96,138,832,262","158,902,056,650","805,990,365,613","108,361,668,359","1,265,342,956,063","124,082","174,948,305","108,536,616,664","8,410,142,182",0,"251,683,344","99,874,791,138","181,392,990,148","281,267,781,286"
8,"16,989,145","1,938,392,665,752","10,384,405","230,523,789,899","6,601,882","175,582,269,267","178,799,675,324","1,354,716,381,497","193,248,112,439","1,826,718,240,155","341,108","257,914,284","193,506,026,724","7,755,958,358",0,"42,122,627","185,707,945,739","267,524,994,528","453,232,940,266"
9,"16,989,970","4,745,335,816,489","4,783,210","108,807,352,802","12,206,401","406,105,081,105","170,075,137,894","4,061,083,705,696","885,349,865,208","4,503,729,898,954","4,789,303","16,305,897,124","901,655,762,332","505,464,806","8,774,049,698","2,984,307","909,921,362,917","434,372,680,793","1,344,294,043,710"


#### Differences Table

The differences table displays the difference between your baseline and refoms. You can also group the results by decile or income bin.

In [15]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

,tax_cut,tax_inc,count,mean,tot_change,perc_inc,perc_cut,share_of_change,perc_aftertax
0,0,"83,321","16,988,849",0,"1,311,238",0.49%,0.00%,-0.00%,0.00%
1,"1,421,766","46,416","16,990,011",-11,"-181,207,431",0.27%,8.37%,0.16%,-0.10%
2,"5,131,077","66,225","16,989,299",-124,"-2,105,507,033",0.39%,30.20%,1.82%,-0.80%
3,"8,014,836","679,403","16,988,882",-231,"-3,925,229,714",4.00%,47.18%,3.39%,-1.06%
4,"10,835,681","973,989","16,989,771",-432,"-7,339,844,752",5.73%,63.78%,6.35%,-1.51%
5,"13,056,241","1,092,714","16,989,771",-662,"-11,243,277,317",6.43%,76.85%,9.72%,-1.77%
6,"14,549,172","1,309,830","16,989,779",-976,"-16,579,363,872",7.71%,85.63%,14.34%,-2.01%
7,"15,371,288","1,382,683","16,988,988","-1,142","-19,408,562,984",8.14%,90.48%,16.79%,-1.74%
8,"16,107,307","786,055","16,989,812","-1,544","-26,227,089,786",4.63%,94.81%,22.68%,-1.64%
9,"11,904,943","1,797,575","16,989,678","-1,685","-28,620,693,548",10.58%,70.07%,24.75%,-0.65%


#### Plotting

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [16]:
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [17]:
mtr_plot_data = mtr_graph_data(calc, calc_response)

In [18]:
show(xtr_graph_plot(mtr_plot_data))

In [19]:
atr_plot_data = atr_graph_data(calc, calc_response)

In [20]:
show(xtr_graph_plot(atr_plot_data))

#### Multiyear diagnostic tables

You can also produce diagnostic tables up to 2026

In [21]:
multiyear_diagnostic_table(calc, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,557.3","10,992.3","11,462.4","11,984.1","12,542.0","13,134.4","13,759.5","14,413.0","15,090.7"
Itemizers (#m),67.9,69.3,70.8,72.4,74.1,75.7,77.4,79.1,80.8
Itemized Deduction ($b),"1,491.9","1,569.1","1,654.6","1,749.0","1,849.7","1,953.3","2,063.5","2,177.7","2,299.5"
Standard Deduction Filers (#m),84.0,84.6,85.2,85.7,86.2,86.7,87.2,87.8,88.3
Standard Deduction ($b),772.4,795.3,818.1,840.8,864.5,888.7,913.2,939.1,965.7
Personal Exemption ($b),"1,221.6","1,266.2","1,312.9","1,361.2","1,411.0","1,462.7","1,516.3","1,572.0","1,629.8"
Taxable Income ($b),"7,484.1","7,786.9","8,115.0","8,484.3","8,881.4","9,307.6","9,758.6","10,230.9","10,717.4"
Regular Tax ($b),"1,402.3","1,462.7","1,528.1","1,602.7","1,683.1","1,769.9","1,862.4","1,958.6","2,056.4"
AMT Income ($b),"9,722.1","10,112.9","10,534.0","11,001.4","11,501.7","12,034.5","12,595.8","13,183.4","13,790.9"


In [22]:
multiyear_diagnostic_table(calc_response, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,529.9","10,963.6","11,432.4","11,952.4","12,508.7","13,099.3","13,722.7","14,374.5","15,050.4"
Itemizers (#m),28.8,24.7,20.5,17.0,17.9,18.7,19.6,20.6,21.6
Itemized Deduction ($b),799.3,663.8,528.7,413.3,449.0,485.5,525.2,568.2,614.4
Standard Deduction Filers (#m),123.1,129.2,135.5,141.1,142.4,143.7,145.0,146.3,147.6
Standard Deduction ($b),"2,120.9","2,289.3","2,469.6","2,646.0","2,729.2","2,816.3","2,905.5","2,997.2","3,091.4"
Personal Exemption ($b),"1,221.6","1,266.2","1,312.9","1,361.2","1,411.0","1,462.7","1,516.3","1,572.0","1,629.8"
Taxable Income ($b),"7,101.9","7,477.4","7,874.0","8,304.4","8,713.1","9,149.7","9,612.6","10,097.0","10,597.6"
Regular Tax ($b),"1,289.5","1,367.3","1,448.9","1,537.2","1,617.8","1,704.5","1,796.8","1,893.0","1,991.3"
AMT Income ($b),"10,038.0","10,504.2","11,016.9","11,587.7","12,112.0","12,669.9","13,257.7","13,871.0","14,505.2"


#### Reporting your changes

To display what reforms you included in a way that is easy for humans to read, you can use the `reform_documentation` method. It will print out all of the policy parameters you've specified with a short description and their default and current values


_Note: this feature is not yet available in the taxcalc package. You must use the source code to access it. It will be available in the next taxcalc release._

In [23]:
print(calc_response.reform_documentation(reforms))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _ID_RealEstate_hc : [ 0.25]
  name: State, local, and foreign real estate taxes deduction haircut.
  desc: This decimal fraction reduces real estate taxes paid eligible to
        deduct in itemized deduction.
  baseline_value: 0.0
 _ID_StateLocalTax_hc : [ 0.25]
  name: State and local income and sales taxes deduction haircut.
  desc: This decimal fraction reduces the state and local income and sales tax
        deduction.
  baseline_value: 0.0
 _II_rt1 : [ 0.12]
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 1
  desc: The lowest tax rate, applied to the portion of taxable income below
        tax bracket 1.
  baseline_value: 0.1
 _II_rt2 : [ 0.12]
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 2
  desc: The second lowest tax rate, applied to the portion of taxable income
    

One thing I didn't cover is how to extract the marginal tax rates. All you need to do is call the `Calculator.mtr()` method. It will return MTR for individual income tax, payroll tax, and individaul income tax + payroll tax

In [24]:
mtr_payroll, mtr_income, mtr_combined = calc.mtr()

In [26]:
mtr_payroll

array([ 0.14212726,  0.14212726,  0.14212726, ...,  0.14212726,
        0.14212726,  0.14212726])

In [27]:
mtr_income

array([ 0.13934046,  0.        ,  0.        , ...,  0.23687877,
        0.23687877,  0.23687877])

In [28]:
mtr_combined

array([ 0.28146772,  0.14212726,  0.14212726, ...,  0.37900604,
        0.37900604,  0.37900604])